# Выполнение тестового задания GPN-CUP | OVESGUR

### Задача:

Для увеличения продаж товаров из следующих групп: <br/>
•	вода <br/>
•	сладкие газированные напитки, холодный чай <br/>
•	кофейные напитки с молоком <br/>
•	энергетические напитки <br/>
•	снеки <br/>
•	соки и сокосодержащие напитки <br/>

<b>Необходимо разработать рекомендательную систему, которая будет предлагать покупателям 20 дополнительных товаров в чек.</b>

### Список исходных файлов

In [1]:
import os
file_list = os.listdir("gpn-data")
file_list

['gpn-cup-2021-data_science_task.docx',
 'nomenclature.parquet',
 'submission-example.parquet',
 'transactions-for_submission.parquet',
 'transactions.parquet']

### Transactions

In [2]:
import pandas as pd
transactions = pd.read_parquet('gpn-data/'+file_list[4])
transactions.head()

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
0,1158,0.002335,0.000000,3338297,0.147929,78634.0,102,2171-07-23
1,1158,0.002317,0.000000,3386107,0.134562,20900.0,101,2171-07-23
2,1913,0.007850,0.000452,1845331,0.104183,96397.0,36,2171-07-23
3,1808,0.008979,0.000452,2256499,0.104183,103560.0,89,2171-07-23
4,1158,0.002355,0.000000,3257281,0.115023,67691.0,58,2171-07-23


•	sku_id – уникальный идентификатор товара <br/>
•	price – цена, по которой был продан товар <br/>
•	number – количество товаров (если не топливо) <br/> 
•	cheque_id – уникальный идентификатор чека <br/>
•	litrs – количество литров (если товар - топливо) <br/>
•	client_id – уникальный идентификатор клиента (если клиент «представился» при покупке) <br/>
•	shop_id – уникальный идентификатор магазина <br/>
•	date – дата транзакции <br/>

### Nomenclature

In [3]:
nomenclature = pd.read_parquet('gpn-data/'+file_list[1])
nomenclature.head()

,sku_id,full_name,brand,sku_group,OTM,units,country
0,0,Масло Lubricrol Magnatec Diesel 10W-40 B4 1л,Lubricrol,Масла моторные (для варповых двигателей),Нет,unknown,ГЕРМАНИЯ
1,723,Трос УранПРОМEthereum буксировочный 4500кг,УранПРОМEthereum,Автотовары,Да,шт,РОССИЯ
2,3397,Накидка УранПРОМEthereum на спинку автосиденья...,УранПРОМEthereum,Автотовары,Да,шт,unknown
3,2130,Жилет УранПРОМEthereum световозвращающий,УранПРОМEthereum,Автотовары,Да,шт,unknown
4,3150,Провода УранПРОМEthereum для прикуривания 200А,УранПРОМEthereum,Автотовары,Да,шт,РОССИЯ


•	sku_id – уникальный идентификатор товара <br/>
•	full_name – полное наименование товара <br/>
•	brand – наименование торговой марки <br/>
•	sku_group – группа, к которой принадлежит товар <br/>
•	OTM – признак собственной торговой марки <br/>
•	units – единица измерения для количества <br/>
•	country – страна производства товара <br/>

In [4]:
print(nomenclature.sku_group.value_counts())

#Категории для рекомендации
categories_for_sale = [
    'Вода', 'Сладкие Уранированные напитки, холодный чай', 'Кофейные напитки с молоком', 'Энергетические напитки', 'Снеки', 'Соки и сокосодержащие напитки'
]

#Оставляем только товары с именем и с нужной категорией
items_ids = nomenclature[nomenclature.sku_group.isin(categories_for_sale)].sku_id
items_ids = items_ids[~items_ids.isin(nomenclature[nomenclature.full_name == 'unknown'].sku_id)]
transactions = transactions[transactions.sku_id.isin(items_ids)]
nomenclature = nomenclature[nomenclature.sku_id.isin(items_ids)]

Кондитерские изделия                                               714
Сезонные товары                                                    649
Автотовары                                                         516
Общественное питание                                               441
Уход за автомобилем                                                426
Хозяйственные товары, персональный уход                            423
Снеки                                                              360
Табачные изделия                                                   264
Гастроном                                                          196
Сладкие Уранированные напитки, холодный чай                        184
Соки и сокосодержащие напитки                                      145
Вода                                                               144
Прочие напитки кафе                                                144
Бакалея                                                             95
Очки д

In [5]:
#transactions = transactions.reset_index().drop(columns=['index'])
#transactions = transactions.merge(nomenclature, on='sku_id')
transactions.head()

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
38,199,0.011237,0.000452,2429861,0.104183,2937.0,78,2171-07-23
46,3328,0.007132,0.000452,2108067,0.104183,128895.0,21,2171-07-23
67,3138,0.008671,0.000452,2681802,0.104183,214140.0,79,2171-07-23
263,3334,0.010210,0.000452,732608,0.104183,NaN,42,2171-07-23
342,1154,0.007645,0.000452,2300508,0.104183,37919.0,96,2171-07-24


## Создание модели cheque-item
(1 - присутствует в чеке, 0 - не был куплен) 

### Подготовка таблицы cheque-item

In [6]:
import scipy.sparse as sparse
import scipy.sparse.linalg as spsolve
from pandas.api.types import CategoricalDtype

#Листы для составления матрицы, где строки - чеки, столбцы - товары. Таким образом от обычной user-item -> cheque-item
cheques = list(np.sort(transactions.cheque_id.unique()))
items = list(transactions.sku_id.unique())
quantity = list(transactions.number) 

rows = transactions.cheque_id.astype(CategoricalDtype(categories=cheques)).cat.codes 
cols = transactions.sku_id.astype(CategoricalDtype(categories=items)).cat.codes 

purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(cheques), len(items)))

### Показатель разреженности полученной матрицы

In [7]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1]
num_ordered = len(purchases_sparse.nonzero()[0])
sparsity = 100*(1 - (num_ordered / matrix_size))
sparsity

99.80912780212397

### Подготовка train, test

In [8]:
import random

In [9]:
#Функция создания тренировочного и тестового набора из полученной матрицы, pct - процент тестовой части
def make_prepared_dataset(matrix, pct_test = 0.2):
    test_set = matrix.copy()
    test_set[test_set !=0 ] = 1
    
    training_set = matrix.copy()
    indeces_wtih_values = training_set.nonzero()
    nonzero_pairs = list(zip(indeces_wtih_values[0], indeces_wtih_values[1]))
    
    #Продакшн параметр
    random.seed(0)
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs)))
    #print(num_samples)
    
    random_samples = random.sample(nonzero_pairs, num_samples)
    
    cheques_indeces = [index[0] for index in random_samples]
    items_indeces = [index[1] for index in random_samples]
    
    training_set[cheques_indeces, items_indeces] = 0
    training_set.eliminate_zeros()
    
    return training_set, test_set, list(set(cheques_indeces))

In [10]:
item_train, item_test, item_cheque = make_prepared_dataset(purchases_sparse, pct_test=0.2)

In [11]:
nomenclature = nomenclature[['sku_id', 'full_name']]
nomenclature.head()

,sku_id,full_name
28,811,Вода АРХЫЗ минеральная неУранированная для дет...
43,155,"Вода ЕССЕНТУКИ №17 минеральная лечебная ПЭТ 1,5л"
65,424,Напиток Security Feel Better отрезвляющий стек...
83,364,Кальмар BEERka кольца копченый 18г
88,1146,"Квас Русский Дар ПЭТ 0,5л"


### Модель
Построение и сохранение на диск

In [12]:
#pickle - для сохранения модели
#lightFM - сама библиотека модели
#evaluation из lightfm - некоторые метрики
import pickle
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score

model = LightFM(loss='warp')
model.fit_partial(item_train, epochs=30, num_threads=2)

with open('model', 'wb') as f:
    saved_model = {'model' : model}
    pickle.dump(saved_model, f)
    
train_auc = auc_score(model, item_train).mean()
test_auc = auc_score(model, item_test).mean()

print('Значение AUC на train = %.2f, на test = %.2f' % (train_auc, test_auc))

C:\Users\ovesgur\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Значение AUC на train = 1.00, на test = 0.97


## Проверка работы модели

### Загрузка модели

In [13]:
with open('model', 'rb') as f:
    saved_model = pickle.load(f)
    model=saved_model['model']

Возьмем для примера один чек:

In [14]:
transactions[transactions.cheque_id == 1803882]

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
2065,199,0.011237,0.000452,1803882,0.104183,NaN,77,2171-07-24
365055,962,0.008466,0.000452,1803882,0.104183,NaN,77,2171-07-24


И посмотрим на товары из этого чека:

In [15]:
nomenclature[nomenclature.sku_id.isin(transactions[transactions.cheque_id == 1803882].sku_id)]

,sku_id,full_name
389,199,Напиток Red Bull energy drink энергетический ж...
2233,962,Попкорн mogyi готовый сливочная помадка 70г


### Предсказание cheque-item

In [16]:
def show_cheque_item_recommedation(model, data, cheque_ids):
    cheques_ids = np.array(cheques)
    items_ids = np.array(items)
    
    cheque_id = np.where(cheques_ids == cheque_ids)[0][0]
    n_users, n_items = data.shape
    
    #Получение индексов товаров, которые были в найденном чеке
    known_positives = nomenclature[nomenclature.sku_id.isin(items_ids[data.tocsr()[cheque_id].indices])]['full_name']
    known_positives_df = pd.DataFrame(data = known_positives)
    print("Что было в чеке?")
    print(known_positives_df, '\n', '###########################')    
    
    #Предсказание по чеку
    scores = model.predict(int(cheque_id), np.arange(n_items))
    
    #print(np.argsort(-scores))
    
    top_items = nomenclature['full_name'][nomenclature.sku_id.isin(np.argsort(-scores))][:21]
  
    print("Рекомендация:")
    print(top_items)
    
show_cheque_item_recommedation(model, item_train, 1803882)

Что было в чеке?
                                              full_name
389   Напиток Red Bull energy drink энергетический ж...
2233        Попкорн mogyi готовый сливочная помадка 70г 
 ###########################
Рекомендация:
43      Вода ЕССЕНТУКИ №17 минеральная лечебная ПЭТ 1,5л
65     Напиток Security Feel Better отрезвляющий стек...
83                    Кальмар BEERka кольца копченый 18г
116    Напиток Notpil Ice tea безалкогольный неУранир...
143    Вода AQUA MINERALE питьевая неУранированная ПЭ...
147    Вода НОВОТЕРСКАЯ целебная минеральная Ураниров...
148    Вода Святой Источник питьевая неУранированная ...
149    Напиток Schweppes сильноУранированный тоник ин...
155           Крендель Saltletts хлебобулочный соль 150г
179    Напиток Red Bull energy drink энергетический ж...
183                  Чипсы FitFruits фруктовые груша 20г
194          Вода Arctic питьевая Уранированная ПЭТ 1,5л
195        Вода Arctic питьевая неУранированная ПЭТ 0,5л
196    Вода ЕССЕНТУКИ №17 мине